In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnext50_32x4d

from tqdm import tqdm
import json

In [2]:
data_folder = ImageFolder('img')

## OLD

In [29]:
category_file = 'notation/class2idx.json'
with open(category_file) as f:
    class2idx = json.load(f)

# convert to label -> folder name
idx2class = {v: k for k, v in dataset.dataset.class_to_idx.items()}
# mapping folder name into original index
new_idxs = {k: class2idx[v] for k, v in idx2class.items()}

In [3]:
trans = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [30]:
class DeepFashion(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return 50000

    def __getitem__(self, idx):
        img, lbl = self.dataset[idx]
        lbl = new_idxs[lbl]
        return self.transform(img), lbl
dataset = DeepFashion(data_folder, transform=trans)

## NEW

In [39]:
category_file = 'notation/class2idx.json'
with open(category_file) as f:
    categories = json.load(f)

In [39]:
trans = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class2idx = {i: c for c, i in data_folder.class_to_idx.items()}
idx2idx = {i: categories[c] for i, c in class2idx.items()}

In [45]:
dataset = ImageFolder('img', transform=trans,
                      target_transform=lambda x: idx2idx[x])

In [46]:
train_data = DataLoader(dataset, batch_size=64, num_workers=8, pin_memory=True)

In [47]:
def train(model, data, epoch, criterion, optimizer, device):
    model.train()
    print('==========Train Epoch {}=========='.format(epoch))
    loss_list = []
    acc_count = 0

    for i, (image, label) in tqdm(enumerate(data), total=len(data)):
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        score = model(image) # predict the label
        loss = criterion(score, label) # calculate error
        loss_list.append(loss.item())
        pred = torch.argmax(score, dim=1)
        correct = pred.eq(label)
        acc_count += correct.sum().item()
        
        loss.backward()  # back-propagation
        optimizer.step() # gradient descent

    acc = acc_count / (len(data) * data.batch_size) * 100
    return sum(loss_list) / len(loss_list), acc

def test(model, data, criterion, device):
    model.eval()
    loss_list = []

    acc_count = 0
    for i, (image, label) in tqdm(enumerate(data), total=len(data)):
        image = image.to(device)
        label = label.to(device)

        score = model(image)
        loss = criterion(score, label)
        loss_list.append(loss.item())

        pred = torch.argmax(score, dim=1)
        correct = pred.eq(label)
        acc_count += correct.sum().item()

    acc = acc_count / (len(data) * data.batch_size) * 100
    print('----------Acc: {}%----------'.format(acc))
    return sum(loss_list) / len(loss_list), acc

In [48]:
model = resnext50_32x4d(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 50)

In [49]:
device = torch.device('cuda')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)

In [56]:
%%capture output
# Hyper Parameters
max_epochs = 10
log_interval = 1

train_acc_list = []
train_loss_list = []
# test_acc_list = []
# test_loss_list = []

for epoch in range(1, max_epochs + 1):
    train_acc, train_loss = train(model, train_data, epoch, criterion, optimizer, device)
#     test_acc, test_loss = test(model, data, criterion, device)

    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
#     test_acc_list.append(test_acc)
#     test_loss_list.append(test_loss)
    if epoch % log_interval == 0:
        print('=' * 20, 'Epoch', epoch, '=' * 20)
        print('Train Acc: {:.6f} Train Loss: {:.6f}'.format(train_acc, train_loss))
#         print('Test Acc: {:.6f} Test Loss: {:.6f}'.format(test_acc, test_loss))

RuntimeError: CUDA error: unspecified launch failure